In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv('labeled_data.csv')
df.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [3]:
df.drop(columns=['Unnamed: 0','count','hate_speech','offensive_language','neither'],inplace=True)
df.head()

,class,tweet
0,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [4]:
df.isna().sum()

class    0
tweet    0
dtype: int64

In [5]:
df['tweet'][0]

"!!! RT @mayasolovely: As a woman you shouldn't complain about cleaning up your house. &amp; as a man you should always take the trash out..."

In [6]:
df['tweet'][100]

'"@ClicquotSuave: LMAOOOOOOOOOOO this nigga @Krillz_Nuh_Care http://t.co/AAnpSUjmYI" &lt;bitch want likes for some depressing shit..foh'

In [7]:
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
import re

In [8]:
## removing special characters
df['reviewText']=df['tweet'].apply(lambda x:re.sub('[^a-z A-z 0-9-]+','',x))
## removing the stopwords
df['reviewText']=df['reviewText'].apply(lambda x:" ".join([y for y in x.split() if y not in set(stopwords.words('english'))]))
## removing url
df['reviewText']=df['reviewText'].apply(lambda x:re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?','',str(x)))
## removing html tags
df['reviewText'] = df['reviewText'].apply(lambda x: BeautifulSoup(x, 'html.parser').get_text())
## remove any additional spaces
df['reviewText']=df['reviewText'].apply(lambda x:" ".join(x.split()))

In [9]:
from nltk.stem import WordNetLemmatizer

In [10]:
lemmatizer=WordNetLemmatizer()

In [11]:
def lemmatize_text(text):
    return " ".join(lemmatizer.lemmatize(word) for word in text.split())

In [12]:
df['lemmatized_text']=df['reviewText'].apply(lambda x:lemmatize_text(x))

In [13]:
df.head()

,class,tweet,reviewText,lemmatized_text
0,2,!!! RT @mayasolovely: As a woman you shouldn't...,RT mayasolovely As woman shouldnt complain cle...,RT mayasolovely As woman shouldnt complain cle...
1,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,RT mleew17 boy dats coldtyga dwn bad cuffin da...,RT mleew17 boy dat coldtyga dwn bad cuffin dat...
2,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,RT UrKindOfBrand Dawg RT 80sbaby4life You ever...,RT UrKindOfBrand Dawg RT 80sbaby4life You ever...
3,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,RT C_G_Anderson viva_based look like tranny,RT C_G_Anderson viva_based look like tranny
4,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,RT ShenikaRoberts The shit hear might true mig...,RT ShenikaRoberts The shit hear might true mig...


In [14]:
df.drop(columns=['tweet','reviewText'],inplace=True)

In [15]:
df.head()

,class,lemmatized_text
0,2,RT mayasolovely As woman shouldnt complain cle...
1,1,RT mleew17 boy dat coldtyga dwn bad cuffin dat...
2,1,RT UrKindOfBrand Dawg RT 80sbaby4life You ever...
3,1,RT C_G_Anderson viva_based look like tranny
4,1,RT ShenikaRoberts The shit hear might true mig...


In [16]:
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split

In [17]:
vocab_size=1000
one_hot_representation=[one_hot(words,vocab_size) for words in df['lemmatized_text']]

In [18]:
one_hot_representation

[[457, 585, 59, 72, 405, 474, 177, 121, 361, 408, 910, 541, 266],
 [457, 737, 156, 535, 716, 431, 250, 947, 535, 471, 73, 872],
 [457, 787, 385, 457, 96, 469, 611, 232, 222, 529, 504, 469, 117, 694],
 [457, 701, 678, 978, 610, 172, 728, 969, 41],
 [457, 548, 487, 694, 599, 481, 738, 481, 264, 222, 666, 957, 217],
 [725, 944, 972, 487, 694, 972, 779, 708, 902, 153, 732, 471, 649],
 [578, 149, 81, 196, 873, 222, 149, 699, 780, 694, 657],
 [582, 135, 760, 442, 6, 222, 717, 738, 460, 705],
 [361, 481, 545, 957, 222, 58, 361, 790],
 [353, 916, 483, 280, 479],
 [711, 222, 192, 526, 290, 149, 500, 314, 969, 802],
 [882, 366, 222, 366, 764],
 [184, 471, 196, 401, 419, 625, 717],
 [250, 222, 42, 969],
 [222, 545],
 [222, 119, 929],
 [222, 80, 58],
 [222, 785],
 [222, 545, 777, 875, 389],
 [371, 400, 361, 250, 222],
 [10, 222, 287, 193, 680],
 [392, 222, 969, 600],
 [287, 196, 471, 262, 441],
 [232, 222, 502, 60, 420, 382, 649, 340, 194, 172, 232, 717],
 [699, 957, 222, 254, 672, 279, 910, 223, 

In [19]:
df['lemmatized_text'].iloc[0]

'RT mayasolovely As woman shouldnt complain cleaning house amp man always take trash'

In [ ]:
sentence_length=20
embedded_tweet=pad_sequences(one_hot_representation,padding='pre',maxlen=sentence_length)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (24783,) + inhomogeneous part.

In [21]:
embedded_tweet

array([[  0,   0,   0, ..., 910, 541, 266],
       [  0,   0,   0, ..., 471,  73, 872],
       [  0,   0,   0, ..., 469, 117, 694],
       ...,
       [  0,   0,   0, ..., 381, 532, 790],
       [  0,   0,   0, ..., 222, 655, 585],
       [  0,   0,   0, ..., 413,  44, 938]])

In [22]:
X=np.array(embedded_tweet)
y=np.array(df['class'])

In [23]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [24]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Embedding,LSTM

In [25]:
dimension=50
model=Sequential()
model.add(Embedding(vocab_size,dimension,input_length=sentence_length))
model.add(LSTM(100,return_sequences=True))
model.add(LSTM(50,return_sequences=True))
model.add(LSTM(50))
model.add(Dense(vocab_size,activation="softmax"))

In [26]:
model.compile(optimizer="adam",loss="sparse_categorical_crossentropy",metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 50)            50000     
                                                                 
 lstm (LSTM)                 (None, 20, 100)           60400     
                                                                 
 lstm_1 (LSTM)               (None, 20, 50)            30200     
                                                                 
 lstm_2 (LSTM)               (None, 50)                20200     
                                                                 
 dense (Dense)               (None, 1000)              51000     
                                                                 
Total params: 211800 (827.34 KB)
Trainable params: 211800 (827.34 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [28]:
history=model.fit(X_train,y_train,epochs=100,validation_data=[X_test,y_test],batch_size=32)

Epoch 1/100
620/620 [==============================] - 28s 45ms/step - loss: 0.8350 - accuracy: 0.7746 - val_loss: 0.6706 - val_accuracy: 0.7730
Epoch 2/100
620/620 [==============================] - 26s 43ms/step - loss: 0.6660 - accuracy: 0.7746 - val_loss: 0.6677 - val_accuracy: 0.7730
Epoch 3/100
620/620 [==============================] - 23s 38ms/step - loss: 0.6645 - accuracy: 0.7746 - val_loss: 0.6660 - val_accuracy: 0.7730
Epoch 4/100
620/620 [==============================] - 27s 43ms/step - loss: 0.6642 - accuracy: 0.7746 - val_loss: 0.6657 - val_accuracy: 0.7730
Epoch 5/100
620/620 [==============================] - 38s 61ms/step - loss: 0.6639 - accuracy: 0.7746 - val_loss: 0.6683 - val_accuracy: 0.7730
Epoch 6/100
620/620 [==============================] - 28s 44ms/step - loss: 0.6644 - accuracy: 0.7746 - val_loss: 0.6677 - val_accuracy: 0.7730
Epoch 7/100
620/620 [==============================] - 24s 38ms/step - loss: 0.6647 - accuracy: 0.7746 - val_loss: 0.6654 - val_ac

In [32]:
pred=np.argmax(model.predict(X_test),axis=-1)

155/155 [==============================] - 1s 9ms/step


In [33]:
y_test[:5]

array([1, 1, 2, 1, 1], dtype=int64)

In [34]:
pred[:5]

array([1, 1, 2, 1, 1], dtype=int64)

In [35]:
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

In [37]:
print(classification_report(y_test,pred))

              precision    recall  f1-score   support

           0       0.25      0.26      0.25       290
           1       0.90      0.89      0.89      3832
           2       0.68      0.70      0.69       835

    accuracy                           0.82      4957
   macro avg       0.61      0.62      0.61      4957
weighted avg       0.82      0.82      0.82      4957



In [38]:
model.save("hate_speech_model.h5")

c:\Users\KIIT0001\Desktop\DESKTOP_OWN\CODING\venv\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
